


Kuo Yang
Modified 4/18/25
FAERS Test
data source: FDA Adverse Events Reporting System (FAERS, from FDA.gov)
filtered for semaglutide, all cases, accessed 3/22/25

Disclaimer: This is for academic coursework ONLY. Do not use for clinical decision making; no causations implied in analysis. See end of document for full disclosure/disclaimer.

In [ ]:
import numpy as np
import pandas as pd
#smgtdf=pd.read_csv('smgt.csv')
smgtdf = pd.read_excel('smglt.xlsx')
lrgtdf = pd.read_excel('LGTD.xlsx')

In [ ]:
pd.set_option('display.max_columns', None)

#DF meta check
#print("Dimensions are:",smgtdf.shape,'\n***********')
#print("Column Names:",smgtdf.columns.to_list())
#smgtdf.head(5)
#meow function - dataframe QC
def meow(x):
    print("Dimensions are:",x.shape,'\n***********')
    print("Column Names:",x.columns.to_list())
    print(x.head(5))
meow(smgtdf)
meow(lrgtdf)

In [ ]:
varsel = list(['Case ID','Suspect Product Active Ingredients','Serious','Outcomes','Sex','Patient Age','Patient Weight','Reporter Type', 'Country where Event occurred','Compounded Flag'])
smgtdfa = smgtdf[varsel]
lrgtdfa = lrgtdf[varsel]
meow(smgtdfa)
meow(lrgtdfa)

def meow2(df,clmn):
    for i in clmn:
        col1 = df[i].unique()
        print('unique values of',i,'=',col1,'\n',"\U0001F408"*20)

varsel2 = list(['Sex','Suspect Product Active Ingredients','Serious','Reporter Type','Country where Event occurred','Compounded Flag'])
meow2(df=smgtdfa,clmn=varsel2)
meow2(df=lrgtdfa,clmn=varsel2)

#combine semaglutide+liraglutide
glp1df = pd.concat([smgtdfa,lrgtdfa])
print('GLP-1 Combined Data Frame \n', '*'*50)
glp1df.head()

In [ ]:
glp1df['sex2'] = glp1df['Sex'].map({'Female':0,'Male':1,'Not Specified':3})
glp1df['serious2'] = glp1df['Serious'].map({'Non-Serious':0,'Serious':1})
glp1df['reporter2'] = glp1df['Reporter Type'].map({'Healthcare Professional':1,'Consumer':2,'Not Specified':3,'Other':4})
glp1df['cmpndf'] = np.where(glp1df['Compounded Flag']== 'Yes', 1, 0)

In [ ]:
glp1df.head()

In [ ]:
#filter for only semaglutide and liraglutide API
# Add a new column named 'Price'
glp1df['api'] = [1 if x == 'Semaglutide' else 2 if x== 'Liraglutide' else 3 for x in glp1df['Suspect Product Active Ingredients']]
glp1df2 = glp1df[glp1df['api'].isin([1,2])]
glp1df2.head()

In [ ]:
import hvplot.pandas
import panel as pn

In [ ]:
frqt = glp1df['Serious'].value_counts()
print(frqt,'\n*******')

#Using groupby to count frequencies across all combinations; this is data pipeline
frqt2 = glp1df2.groupby(['Serious', 'Sex']).size().reset_index(name='Count')
print(frqt2)
print(frqt2.columns)


#Multi-level cross-tabulation
#print("\nMulti-level cross-tabulation (Susp Pro API, Serious, Sex, Compounded Flag)")
#frqt_mt = pd.crosstab([glp1df2['Sex'],glp1df2['Compounded Flag'],glp1df2['Suspect Product Active Ingredients']], glp1df2['Serious'])
#print(frqt_mt)


#frqt2_multi_index = frqt2.query("yr<=80").groupby(['yr', 'origin']).mean(numeric_only=True)
#autompg_multi_index.head()

#frqt3 = frqt2.reset_index()
#print(frqt3)
#print(frqt3.columns)
#frqt3.head()
#frqt3m = frqt3.melt(id_vars=['Serious'],value_vars=['Female','Male','Not Specified'],var_name='Sex',value_name='Counts')
#print(frqt3m)

In [ ]:
#make data pipeline interactive (make frqt2 interactive)
idf = glp1df2.interactive()

#Define Panel Widgets
api_w = pn.widgets.ToggleGroup(name='1=semaglutide, 2=liraglutide',options=[1,2],
                               value=[1,2],button_type='success')

#combine pipeline and widgets
ipipeline = (
    idf[
    (idf.api.isin(api_w))
    ]
    .groupby(['Serious','Sex']).size().reset_index(name='Count')
)

ibarplot1 = ipipeline.hvplot.bar(x='Serious',y='Count',by='Sex',stacked=True,color=['hotpink','lightskyblue','mediumpurple'],
                                 xlabel='Seriousness',ylabel='Number of Events',title='GLP-1 Adverse Events',ylim=(0,40000))
ibarplot1

In [ ]:
pn.extension('tabulator')
itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10)
itable

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='GLP-1 Adverse Events', sidebar=["Drug, 1=semaglutide,2=liraglutide",api_w], main=[ibarplot1.panel(),itable.panel()]
)
template.servable()

In [ ]:
frqt2.head()

In [ ]:
#frqt.plot(kind='bar',title='Serious Reactions')
frqt2.hvplot.bar(x='Serious',y='Count',by='Sex',stacked=True,title='GLP-1 Adverse Events',color=['hotpink','lightskyblue','mediumpurple'],
                 xlabel='Seriousness',ylabel='Number of Events')

In [ ]:
frqt2.plot.bar(rot=0, stacked=True)


In [ ]:
import hvplot.pandas
print(frqt)

In [ ]:
frqt.hvplot.bar(x="Serious")

In [ ]:
print('frqt2:\n',frqt2)
print(frqt2.columns)
print('frqt3 data frame:')
print(frqt3,'\n --*5')
print('frqt3m data frame:')
print(frqt3m)

#frqt3 select specific olumns
frqt3temp=frqt3[['Female','Male']]
print(frqt3temp)

In [ ]:
import hvplot.pandas
import holoviews as hv
import numpy as np
import pandas as pd
import panel as pn

pn.extension()

# Checkbox widget for selecting columns
checkbox = pn.widgets.CheckBoxGroup(name='Sex:', options=list(frqt3.columns[1:]), 
                                    value=['Male','Female','Not Specified'])

# Function to plot selected columns
def plot(columns):
    if not columns:
        return pn.pane.Markdown("**Please select at least one column.**")
    return frqt3[columns].hvplot.bar(title="semaglutide Adverse Events",stacked=True,
                                     color=['hotpink', 'lightskyblue', 'mediumpurple'],xlabel='Seriousness',
                                     ylabel='Number of Events',legend='top_right'
                                    )

# Bind the function to the widget correctly
bound_plot = pn.bind(plot, columns=checkbox)

# Layout: checkboxes on top, plot below
layout = pn.Column("## Select Columns to Plot", checkbox, bound_plot)

# Show in notebook or serve as app
layout.servable()

In [ ]:
frqt3.hvplot.bar(x="Serious",title="Selected Columns",stacked=True)
#print(list(frqt3.columns)[1:])

In [ ]:
bar_plot=frqt3.hvplot.bar(x="Serious",stacked=True,color=['hotpink', 'lightskyblue', 'mediumpurple'],xlabel='Category',
                         ylabel='Number of Events',legend='top_right',title='Semaglutide Adverse Events')
component = pn.panel(bar_plot, sizing_mode="stretch_width", )
print(component)
component.servable()

# Create a Panel app
app = pn.Column(
    pn.pane.Markdown("# Interactive Bar Graph"),
    pn.pane.Markdown("A simple bar graph created with hvPlot and served with Panel."),
    bar_plot
)

# Serve the app
app.servable()

In [ ]:
#Test area
import pandas as pd
import numpy as np
import hvplot.pandas
import panel as pn
import holoviews as hv
from holoviews import opts

# Enable extensions
pn.extension('hvplot')
hv.extension('bokeh')

# Create sample data with multiple categories
data = {
    'category': ['A', 'B', 'C', 'D', 'E'] * 3,
    'values': [23, 48, 15, 37, 29, 31, 42, 19, 33, 27, 25, 51, 12, 39, 22],
    'group': ['Group1', 'Group1', 'Group1', 'Group1', 'Group1', 
              'Group2', 'Group2', 'Group2', 'Group2', 'Group2',
              'Group3', 'Group3', 'Group3', 'Group3', 'Group3']
}
df = pd.DataFrame(data)

# Create checkboxes for each group
group1_checkbox = pn.widgets.Checkbox(name='Group1', value=True)
group2_checkbox = pn.widgets.Checkbox(name='Group2', value=True)
group3_checkbox = pn.widgets.Checkbox(name='Group3', value=True)

# Create checkbox for showing values on bars
show_values = pn.widgets.Checkbox(name='Show Values', value=False)

# Create checkbox for sorting bars
sort_bars = pn.widgets.Checkbox(name='Sort Bars', value=False)

# Create a function that updates the plot based on checkbox selections
@pn.depends(group1=group1_checkbox, group2=group2_checkbox, group3=group3_checkbox, 
            show_val=show_values, sort=sort_bars)
def create_bar_plot(group1, group2, group3, show_val, sort):
    # Filter data based on checkbox selections
    groups_to_include = []
    if group1:
        groups_to_include.append('Group1')
    if group2:
        groups_to_include.append('Group2')
    if group3:
        groups_to_include.append('Group3')
    
    filtered_df = df[df['group'].isin(groups_to_include)]
    
    # If no groups selected, return empty plot with message
    if len(filtered_df) == 0:
        return hv.Text(0, 0, 'Please select at least one group').opts(
            width=600, height=400, text_font_size='20pt')
    
    # Sort if requested
    if sort:
        # Group by category and sum values, then sort
        plot_df = filtered_df.groupby('category')['values'].sum().reset_index()
        plot_df = plot_df.sort_values('values', ascending=False)
    else:
        # Group by category and sum values
        plot_df = filtered_df.groupby('category')['values'].sum().reset_index()
    
    # Create the bar plot
    bar_plot = plot_df.hvplot.bar(
        x='category', 
        y='values',
        title='Interactive Bar Chart',
        color='category',
        cmap='Category10',
        height=400,
        width=600
    )
    
    # Add value labels if requested
    if show_val:
        labels = hv.Labels(
            {
                'x': plot_df['category'],
                'y': plot_df['values'],
                'text': [str(int(val)) for val in plot_df['values']]
            },
            ['x', 'y'], 'text'
        ).opts(
            opts.Labels(text_font_size='10pt', text_color='black', 
                       text_align='center', text_baseline='bottom',
                       y_offset=5)
        )
        bar_plot = bar_plot * labels
    
    return bar_plot

# Create a panel with the checkboxes and plot
checkbox_group = pn.Column(
    pn.pane.Markdown("### Filter by Group"),
    group1_checkbox,
    group2_checkbox,
    group3_checkbox,
    pn.Spacer(height=20),
    pn.pane.Markdown("### Options"),
    show_values,
    sort_bars
)

# Create the main layout
main_layout = pn.Row(
    checkbox_group,
    create_bar_plot,
    sizing_mode='stretch_width'
)

# Create the app with a title
app = pn.Column(
    pn.pane.Markdown("# Interactive Bar Chart with Checkboxes"),
    pn.pane.Markdown("Use the checkboxes to filter data and customize the visualization."),
    main_layout,
    sizing_mode='stretch_width'
)

# Make it servable
app.servable()

# Display in notebook
app

In [ ]:
import pandas as pd
import hvplot.pandas
import panel as pn

pn.extension

# Sample DataFrame
df = pd.DataFrame({
    'A': [3, 5, 2],
    'B': [4, 1, 7],
    'C': [6, 3, 5]
}, index=['Item 1', 'Item 2', 'Item 3'])

# Checkbox widget for selecting columns
checkbox = pn.widgets.CheckBoxGroup(name='Select Columns', options=list(df.columns), value=['A'])

# Function to plot selected columns
def plot(columns):
    if not columns:
        return pn.pane.Markdown("**Please select at least one column.**")
    return df[columns].hvplot(kind='bar', title="Selected Columns")

# Bind the function to the widget correctly
bound_plot = pn.bind(plot, columns=checkbox)

# Layout: checkboxes on top, plot below
layout = pn.Column("## Select Columns to Plot", checkbox, bound_plot)

# Show in notebook or serve as app
layout.servable()


# All Data from FDA Adverse Events Reporting System (FAERS)
While the FAERS dashboard offers stakeholders many more ways of searching for and organizing data on adverse events reported to the FDA for many drug and biologic products, there remain limitations to the data. For example, while FAERS contains reports on a particular drug or biologic, this does not mean that the drug or biologic caused the adverse event. Importantly, the FAERS data by themselves are not an indicator of the safety profile of the drug or biologic. Some additional limitations to note include:

Duplicate and incomplete reports are in the system: There are many instances of duplicative reports and some reports do not contain all the necessary information.
Existence of a report does not establish causation: For any given report, there is no certainty that a suspected drug caused the event.  While consumers and healthcare professionals are encouraged to report adverse events, the event may have been related to the underlying disease being treated, or caused by some other drug being taken concurrently, or occurred for other reasons. The information in these reports reflects only the reporter's observations and opinions.
Information in reports has not been verified: Submission of a report does not mean that the information included in it has been medically confirmed nor it is an admission from the reporter that the drug caused or contributed the event.
Rates of occurrence cannot be established with reports: The information in these reports cannot be used to estimate the incidence (occurrence rates) of the events reported.
Patients should talk to their doctor before stopping or changing how they take their medications.
Improving data access and transparency are core concepts that drove the development of this FAERS Dashboard.  The FDA anticipates that this increased transparency will help to spur the submission of more detailed and complete reports from consumers, health care providers and other members of the public. Complete and detailed reports are immensely helpful to the agency when identifying safety signals and choosing particular products for further scrutiny.

In [ ]:
#test area
import pandas as pd
import hvplot.pandas
import holoviews as hv

# Sample dataframe
data = {'category': ['A', 'A', 'B', 'B', 'C', 'C'],
        'value': [10, 15, 20, 25, 30, 35]}
df = pd.DataFrame(data)

# Group by category and create bar plots
bar_plots = df.groupby('category')['value'].hvplot.bar()

# Add interactive elements (e.g., checkboxes)
bar_plots.opts(hv.Plot(), tools=['toggle'])

# Display the interactive plot
bar_plots

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Create sample data with 4 categorical variables
np.random.seed(42)
data = {
    'Gender': np.random.choice(['Male', 'Female', 'Other'], 1000),
    'Education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], 1000),
    'Region': np.random.choice(['North', 'South', 'East', 'West', 'Central'], 1000),
    'Age_Group': np.random.choice(['18-25', '26-35', '36-45', '46+'], 1000)
}

df = pd.DataFrame(data)

print("Sample data:")
print(df.head())
print("\n")

# Method 1: Individual frequency tables for each variable
print("Method 1: Individual frequency tables")
for column in df.columns:
    print(f"\nFrequency table for {column}:")
    freq_table = df[column].value_counts().reset_index()
    freq_table.columns = [column, 'Count']
    print(freq_table)
    # Add percentage
    freq_table['Percentage'] = (freq_table['Count'] / freq_table['Count'].sum() * 100).round(2)
    print(freq_table)

# Method 2: Cross-tabulation between two variables
print("\nMethod 2: Cross-tabulation between Gender and Education")
cross_tab = pd.crosstab(df['Gender'], df['Education'])
print(cross_tab)

# Method 3: Cross-tabulation with percentages
print("\nMethod 3: Cross-tabulation with percentages")
cross_tab_pct = pd.crosstab(df['Gender'], df['Education'], normalize='index') * 100
print(cross_tab_pct.round(2))

# Method 4: Multi-level cross-tabulation (3 variables)
print("\nMethod 4: Multi-level cross-tabulation (Gender, Education, Region)")
multi_tab = pd.crosstab([df['Gender'], df['Education']], df['Region'])
print(multi_tab)

# Method 5: Using groupby to count frequencies across all combinations
print("\nMethod 5: Groupby frequency count across all 4 variables")
grouped = df.groupby(['Gender', 'Education', 'Region', 'Age_Group']).size().reset_index(name='Count')
print(grouped.sort_values('Count', ascending=False).head(10))  # Top 10 combinations

# Method 6: Pivot table for 3 variables with aggregation
print("\nMethod 6: Pivot table with Gender, Education, and Region")
pivot = pd.pivot_table(df, index=['Gender', 'Education'], 
                      columns='Region', 
                      values='Age_Group', 
                      aggfunc='count',
                      fill_value=0)
print(pivot)

# Method 7: Visualizing frequencies with heatmap
print("\nMethod 7: Visualizing with heatmap (check the plot)")
plt.figure(figsize=(12, 8))
cross_tab_plot = pd.crosstab(df['Gender'], df['Education'])
sns.heatmap(cross_tab_plot, annot=True, cmap='YlGnBu', fmt='d')
plt.title('Frequency Distribution: Gender vs Education')
plt.tight_layout()
# plt.show()  # Uncomment to display the plot